In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import string

In [2]:
df = pd.read_csv('Ournews')

In [3]:
df.drop(columns= ['Unnamed: 0', 'url', 'date_extracted', 'author'], inplace= True)

In [4]:
df

,source,title,content
0,aawsat,بريطانيا: أربعة محاور لاستراتيجية جديدة تتصدى ...,حدد رئيس الوزراء البريطاني ديفيد كاميرون، اليو...
1,aawsat,الرئيس هادي يعين نايف البكري محافظا لعدن,أصدر الرئيس عبدربه منصور هادي رئيس الجمهورية ا...
2,aawsat,مجلس الأمن يصادق على «الاتفاق النووي».. وإيران...,وافق مجلس الأمن الدولي، اليوم (الاثنين)، على ا...
3,aawsat,«نيران صديقة» أميركية تودي بحياة 14 جنديا أفغانيا,قال مسؤولون أفغان ان 14 جنديا أفغانيا قتلوا في...
4,aawsat,الخارجية الإيطالية: أربعة من مواطنينا مخطوفون ...,أعلنت وزارة الخارجية الايطالية، اليوم (الاثنين...
...,...,...,...
31025,was,عام / وزير الشؤون البلدية يوجه بإنشاء مكاتب لإ...,الرياض 26 شوال 1436 هـ الموافق 11 اغسطس 2015 ...
31026,was,عام / هيئة الإغاثة تصف أوضاع اللاجئين الهاربين...,جدة 26 شوال 1436 هـ الموافق 11 أغسطس 2015 م و...
31027,was,عام / المشرف العام على مركز الملك سلمان للإغاث...,الرياض 26 شوال 1436 هـ الموافق 11 أغسطس 2015 ...
31028,was,رياضي / أخضر الطائرة في مواجهة مرتقبة أمام الج...,الرياض 26 شوال 1436 هـ الموافق 11 أغسطس 2015 ...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31030 entries, 0 to 31029
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   source   31030 non-null  object
 1   title    30963 non-null  object
 2   content  30897 non-null  object
dtypes: object(3)
memory usage: 727.4+ KB


In [6]:
df.duplicated().sum()

604

In [7]:
df.drop_duplicates(inplace = True)

In [8]:
df.duplicated().sum()

0

In [9]:
max(df.content.str.len())

33687.0

In [10]:
min(df.content.str.len())

1.0

In [11]:
df.content.unique()

array(['حدد رئيس الوزراء البريطاني ديفيد كاميرون، اليوم (الاثنين)، ملامح استراتيجية للتصدي للتطرف داخل بريطانيا؛ وهي مسألة اعتبرها كاميرون "صراع جيلنا"، متعهدا خلال خطابه في مدينة بيرمنغهام بالتصدي لهؤلاء الذين ينشرون التطرف بين الشبان المسلمين البريطانيين.\n\n ورسم كاميرون الاطار العام لاستراتيجية مكافحة التطرف التي المقرر ان تنشر كاملة في وقت لاحق هذا العام، والتي تسعى للتصدي لانتشار الأفكار المتطرفة التي يروج لها متشددو تنظيم "داعش".\n\n وحسبما تناقلت وسائل الإعلام البريطانية، فإن خطة رئيس الوزراء ستكون على مدى خمسة أعوام للقضاء على التطرف الداخلي من خلال أربعة محاور، وهي: القضاء على إيديولوجية التطرف، والوقوف في وجه عمليات التجنيد وغسل الأدمغة بالأفكار المتطرفة، وإعادة اصوات الاسلام المعتدل إلى المجتمع البريطاني، والتعامل مع أزمات الهوية التي يواجهها بعض المسلمين المولودين في بريطانيا.\n\n وسيكون من الأهداف الرئيسية للاستراتيجية مكافحة صعود من يطلق عليهم اسم "متطرفي الداخل"، وهو أمر يقول كاميرون انه لا يمكن عمله دون فهم الاسباب التي تجتذب الناس لـ"داعش" والتصدي لها. إذ قال كاميرون 

In [12]:
df.content.isna().sum()

130

In [14]:
df= df.dropna(subset= ['content'])

In [15]:
df.content.isna().sum()

0